In [1]:
import pandas as pd
import numpy as np

from scipy.optimize import curve_fit

import pygama.math.utils as pgu

import matplotlib
# matplotlib.use('Agg') # when running on cori
import matplotlib.pyplot as plt
#plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm


#from scipy import signal as sig



from pygama.flow import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
import psd_analysis as psd

import calibration_jdetwiler as cb


from pygama.dsp.WaveformBrowser import WaveformBrowser


dg = DataGroup('/global/homes/f/fnafis/krstc/krstc.json', load=True)
    
fontsize = 20  
def draw_canvas(plt, fontsize=fontsize, xlabel='xlabel', ylabel='count', title='title'):
    plt.figure(figsize=(20,16))
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.xlabel(xlabel, fontsize=fontsize*1.5)
    plt.ylabel(ylabel, fontsize=fontsize*1.5)
    plt.legend(fontsize=fontsize)
    plt.title(title, fontsize=fontsize*2)
    
fileDB = pd.read_hdf('/global/cfs/cdirs/legend/software/KrSTC/data/fileDB.h5')





ModuleNotFoundError: No module named 'tinydb'

In [4]:
beginning_cycle = 2019
cycle_range_bkg = [2028,2038, 'bkg_run2'] #run 2
#cycle_range_bkg = [2115,2121, 'bkg_run8']
cycle_range_kr = [2039,2058, 'kry_run3'] #gain=1.0
#cycle_range = [2059,2093, 'kry_run4'] #gain>1.0
#cycle_range = [2093,2100, 'kry_run5'] #gain=1.0
#cycle_range = [2147, 2202, 'kry_run12']
#cycle_range = [2203, 2210, 'kry_run13']
#cycle_range = [2280, 2332, 'kry_run17']


runtime_bkg = fileDB['runtime'][cycle_range_bkg[0]-beginning_cycle:cycle_range_bkg[1]-beginning_cycle].sum()
runtime_kr = fileDB['runtime'][cycle_range_kr[0]-beginning_cycle:cycle_range_kr[1]-beginning_cycle].sum()



lh5_dir = dg.lh5_dir
dsp_list = lh5_dir + dg.fileDB['dsp_path'] + '/' + dg.fileDB['dsp_file']


hit_list_kr = dsp_list[cycle_range_kr[0]-beginning_cycle:cycle_range_kr[1]+1-beginning_cycle]
hit_list_bkg = dsp_list[cycle_range_bkg[0]-beginning_cycle:cycle_range_bkg[1]+1-beginning_cycle]



#hit_list = dsp_list[2039-2019]
'''
columns in file:
        ['trapEmax', 'bl', 'bl_sig', 'A_10', 'AoE', 'packet_id', 'ievt', 'energy',
        'energy_first', 'timestamp', 'crate', 'card', 'channel', 'energy_cal',
        'trapEmax_cal']'''


df_hit_kr = lh5.load_dfs(hit_list_kr, ['trapEmax', 'bl', 'bl_sig', 'A_10', 'timestamp', 'ToE', 'bl_slope', 'AoE', 'triEmax'], 'ORSIS3302DecoderForEnergy/dsp')

df_hit_bkg = lh5.load_dfs(hit_list_bkg, ['trapEmax', 'bl', 'bl_sig', 'A_10', 'timestamp', 'ToE', 'AoE', 'triEmax'], 'ORSIS3302DecoderForEnergy/dsp')


#timestamp is in unit of 10ns.
current_pars = [0.430, -0.09979]
df_hit_kr['trapEmax_cal'] = np.polyval(current_pars, df_hit_kr['trapEmax'])
df_hit_bkg['trapEmax_cal'] = np.polyval(current_pars, df_hit_bkg['trapEmax'])

loading data for /global/cfs/cdirs/m2676/data/krstc/LH5/dsp/krstc_run3_cyc2039_dsp.lh5 /global/cfs/cdirs/m2676/data/krstc/LH5/dsp/krstc_run3_cyc2040_dsp.lh5 /global/cfs/cdirs/m2676/data/krstc/LH5/dsp/krstc_run3_cyc2041_dsp.lh5 /global/cfs/cdirs/m2676/data/krstc/LH5/dsp/krstc_run3_cyc2042_dsp.lh5 /global/cfs/cdirs/m2676/data/krstc/LH5/dsp/krstc_run3_cyc2043_dsp.lh5 /global/cfs/cdirs/m2676/data/krstc/LH5/dsp/krstc_run3_cyc2044_dsp.lh5 /global/cfs/cdirs/m2676/data/krstc/LH5/dsp/krstc_run3_cyc2045_dsp.lh5 /global/cfs/cdirs/m2676/data/krstc/LH5/dsp/krstc_run3_cyc2046_dsp.lh5 /global/cfs/cdirs/m2676/data/krstc/LH5/dsp/krstc_run3_cyc2047_dsp.lh5 /global/cfs/cdirs/m2676/data/krstc/LH5/dsp/krstc_run3_cyc2048_dsp.lh5 /global/cfs/cdirs/m2676/data/krstc/LH5/dsp/krstc_run3_cyc2049_dsp.lh5 /global/cfs/cdirs/m2676/data/krstc/LH5/dsp/krstc_run3_cyc2050_dsp.lh5 /global/cfs/cdirs/m2676/data/krstc/LH5/dsp/krstc_run3_cyc2051_dsp.lh5 /global/cfs/cdirs/m2676/data/krstc/LH5/dsp/krstc_run3_cyc2052_dsp.lh5 /gl

In [5]:
df_hit_bkg

,trapEmax,bl,bl_sig,A_10,timestamp,ToE,AoE,triEmax,trapEmax_cal
0,56.083672,58087.003906,4.268143,4.505324,1684985,0.632053,0.080332,35.447865,24.016188
1,651.380127,58085.238281,4.095206,42.752968,1723494,0.604693,0.065634,393.885193,279.993652
2,96.753159,58087.343750,4.362724,7.607142,3843449,0.643706,0.078624,62.280567,41.504070
3,677.044128,58088.144531,4.101443,57.686378,4217343,0.769014,0.085203,520.656250,291.029175
4,702.955688,58088.945312,4.245459,48.063831,4701952,0.639348,0.068374,449.433594,302.171173
...,...,...,...,...,...,...,...,...,...
1339942,544.132080,58082.468750,4.259079,37.140907,155167871886,0.619076,0.068257,336.859222,233.876999
1339943,1825.983887,58084.851562,4.066274,123.451340,155169356191,0.620581,0.067608,1133.171631,785.073303
1339944,255.960571,58084.675781,4.634298,9.414976,155170704031,0.363202,0.036783,92.965385,109.963257
1339945,359.446564,58083.410156,4.232725,20.928213,155171018277,0.540873,0.058223,194.414780,154.462234


In [ ]:

run = '2'

raw_files = []
dsp_files = []

for cyc in np.arange(cycle_range_bkg[0],cycle_range_bkg[1]+1):
    
    
    raw_files.append('/global/cfs/cdirs/legend/data/krstc/LH5/raw/krstc_run'+run+'_cyc'+str(cyc)+'_raw.lh5')
    dsp_files.append('/global/cfs/cdirs/legend/data/krstc/LH5/dsp/krstc_run'+run+'_cyc'+str(cyc)+'_dsp.lh5')




run = '2'
cyc = 2028

#raw_files = '/global/cfs/cdirs/legend/data/krstc/LH5/raw/krstc_run'+run+'_cyc'+str(cyc)+'_raw.lh5'
#dsp_files = '/global/cfs/cdirs/legend/data/krstc/LH5/dsp/krstc_run'+run+'_cyc'+str(cyc)+'_dsp.lh5'

raw_store = lh5.Store()
    
tb_name_raw = 'ORSIS3302DecoderForEnergy/raw'
tb_name_dsp = 'ORSIS3302DecoderForEnergy/dsp'
data_raw, nrows = raw_store.read_object(tb_name_raw, raw_files, idx=x)
    
#bulk_wfs_bkg = data_raw['waveform']['values'].nda[x]


#clockticks = np.arange(0, bulk_wfs_bkg.shape[1], 1)
#time = clockticks/100 #in us


